In [184]:
from scipy import stats
import scipy
import feather as ft
import os
import pandas as pd
import numpy as np
import time
from collections import Counter
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import datetime as dt
import configparser
import re
import utils
import seaborn as sns
from matplotlib_venn import venn3
pd.options.display.max_rows = 20
pd.options.display.max_columns = 50
import warnings
warnings.filterwarnings('ignore')

In [198]:
path = "../../share/data_prod/"
member_profile = ft.read_dataframe(path + 'member_profile.ft')
game_daily_summary = ft.read_dataframe(path +'game_daily_summary.ft').reset_index(drop=True)
member_summary = ft.read_dataframe(path +'member_summary.ft').reset_index(drop=True)
member_profile = ft.read_dataframe(path +'member_profile.ft').reset_index(drop=True)

### data

In [195]:
# %%time
name = "Affiliate"
engine = utils.get_db("IDC", "LckAffiliatedb")
query = "SELECT * FROM [dbo].[{}] WITH (NOLOCK)".format(name)
affiliate = pd.read_sql(query, engine)
# engine.table_names()

In [199]:
c_affiliate = affiliate.copy()
c_affiliate['is_money'] = 1
c_member_summary = member_summary.copy()
c_game_daily_summary = game_daily_summary.copy()
c_member_profile = member_profile.copy()

In [202]:
start_date = '2019-09-01'
end_date = '2019-09-01'

In [218]:
obs_m_p = c_member_profile.query(
    'reg_date >= "{0}" & reg_date <= "{1}" & group_id != 15'.format(start_date, end_date))
obs_m_p_affi_1 = obs_m_p.merge(obs_aff.rename(
    columns={'Id': 'aff_id'}), on='aff_id', how='left').query('is_money == 1')

In [229]:
c_member_profile.head()

,member_id,member_name,status,group_id,balance,reg_date,aff_id,is_aff,is_push_active,country,province,longitude,latitude,gender,birthday,age,zodiac,deposit_days,deposit_oldest_date,deposit_latest_date,deposit_counts,deposit_daily_max,deposit_daily_median,deposit_daily_min,deposit_amount,...,withdraw_amount,withdraw_hour_median,gross_revenue_amount,gross_revenue_max,gross_revenue_min,gross_revenue_std,gross_revenue_median,stake_amount,stake_max,stake_min,stake_std,stake_median,royalty,rebate,bet_counts,bet_oldest_date,bet_latest_date,bet_days,prefer_game,first_game,prefer_game_type,first_game_type,client_type,is_direct,cash_loss
0,1,demormb8888,2,1,0.00,2009-08-11,0,0,True,,,9999.0,9999.0,1.0,1990-02-11,29.0,水瓶,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,NaN,1.0,0.0
1,2,test18rmb,3,1,0.29,2009-08-11,0,0,True,,,9999.0,9999.0,2.0,1981-08-01,38.0,獅子,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,NaN,1.0,0.0
2,3,test18indo1,3,1,0.00,2009-08-13,0,0,True,,,9999.0,9999.0,1.0,1990-02-01,29.0,水瓶,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,NaN,1.0,0.0
3,4,demormb89,2,1,0.00,2009-08-14,0,0,True,,,9999.0,9999.0,1.0,1986-04-15,33.0,牡羊,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,NaN,1.0,0.0
4,5,test9119,3,1,0.00,2009-08-16,0,0,True,,,9999.0,9999.0,1.0,1978-02-03,41.0,水瓶,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,NaN,1.0,0.0


In [227]:
def test_affiliate_mixreport(start_date, end_date):
    summary_for_affi = ''
    obs_aff = c_affiliate.query('affitype == 1 & Status == 1')[['Id', 'is_money']]

    # 資料篩選處理
    obs_m_s = c_member_summary.query(
        'Date >= "{0}" & Date <= "{1}"'.format(start_date, end_date))
    obs_m_s_affi_1 = obs_m_s.merge(obs_aff.rename(
        columns={'Id': 'AffiliateId'}), on='AffiliateId', how='left').query('is_money == 1')
    obs_g_d_s = c_game_daily_summary.query(
        'date >= "{0}" & date <= "{1}"'.format(start_date, end_date))
    obs_g_d_s_affi_1 = obs_g_d_s.merge(obs_aff.rename(
        columns={'Id': 'aff_id'}), on='aff_id', how='left').query('is_money == 1')
    obs_m_p = c_member_profile.query(
        'reg_date >= "{0}" & reg_date <= "{1}" & group_id != 15'.format(start_date, end_date))
    obs_m_p_affi_1 = obs_m_p.merge(obs_aff.rename(
        columns={'Id': 'aff_id'}), on='aff_id', how='left').query('is_money == 1')
    

    # 總存款
    depositAmount_total = obs_m_s.DepositAmount.sum()
    depositAmount_affi_1 = obs_m_s_affi_1.DepositAmount.sum()
    depositAmount_affi_p = depositAmount_affi_1/depositAmount_total

    # 總提款
    withdrawalAmount_total = obs_m_s.WithdrawalAmount.sum()
    withdrawalAmount_affi_1 = obs_m_s_affi_1.WithdrawalAmount.sum()
    withdrawalAmount_affi_p = withdrawalAmount_affi_1/withdrawalAmount_total

    # 存提差(存款-提款)
    deposit_withdrawal_diff_toal = depositAmount_total - withdrawalAmount_total
    deposit_withdrawal_diff_affi_1 = depositAmount_affi_1 - withdrawalAmount_affi_1
    deposit_withdrawal_diff_p = deposit_withdrawal_diff_affi_1/deposit_withdrawal_diff_toal

    # 註冊人數
    registerMember_total = obs_m_p.shape[0]
    registerMember_affi_1 = obs_m_p_affi_1.shape[0]
    registerMember_affi_p = registerMember_affi_1/registerMember_total

    # 註冊後存款人數
    registerAndDepositMember_total = obs_m_p.query(
        'deposit_oldest_date <= "{0}"'.format(end_date)).shape[0]
    registerAndDepositMember_affi_1 = obs_m_p_affi_1.query(
        'deposit_oldest_date <= "{0}"'.format(end_date)).shape[0]
    registerAndDepositMember_affi_p = registerAndDepositMember_affi_1 / \
        registerAndDepositMember_total

    # 總輸贏
    grossRevenuePayout_total = obs_g_d_s.gross_revenue_payout_time.sum()
    grossRevenuePayout_affi_1 = obs_g_d_s_affi_1.gross_revenue_payout_time.sum()
    grossRevenuePayout_affi_p = grossRevenuePayout_affi_1/grossRevenuePayout_total

    # 總流水
    stake_total = obs_g_d_s.stake.sum()
    stake_affi_1 = obs_g_d_s_affi_1.stake.sum()
    stake_affi_p = stake_affi_1/stake_total

    # 活躍人數
    activeMember_total = obs_g_d_s.member_id.nunique()
    activeMember_affi_1 = obs_g_d_s_affi_1.member_id.nunique()
    activeMember_affi_p = activeMember_affi_1/activeMember_total

    # 殺數
    houseEdge_total = obs_g_d_s.gross_revenue_payout_time.sum()/obs_g_d_s.stake.sum()
    houseEdge_affi_1 = obs_g_d_s_affi_1.gross_revenue_payout_time.sum() / \
        obs_g_d_s_affi_1.stake.sum()
    houseEdge_affi_p = houseEdge_affi_1/houseEdge_total

    summary_for_affi = {
        'depositAmount': {
            'name': '总存款',
            'total': depositAmount_total,
            'rabateCash': depositAmount_affi_1,
            'rabateCashPercent': depositAmount_affi_p
        },
        'withdrawalAmount': {
            'name': '总提款',
            'total': withdrawalAmount_total,
            'rabateCash': withdrawalAmount_affi_1,
            'rabateCashPercent': withdrawalAmount_affi_p
        },
        'depositWithdrawalDiff':{
            'name': '存提差',
            'total': deposit_withdrawal_diff_toal,
            'rabateCash': deposit_withdrawal_diff_affi_1,
            'rabateCashPercent': deposit_withdrawal_diff_p
        },
        'registerMember': {
            'name': '注册人数',
            'total': registerMember_total,
            'rabateCash': registerMember_affi_1,
            'rabateCashPercent': registerMember_affi_p
        },
        'registerAndDepositMember': {
            'name': '注册后存款人数',
            'total': registerAndDepositMember_total,
            'rabateCash': registerAndDepositMember_affi_1,
            'rabateCashPercent': registerAndDepositMember_affi_p
        },
        'grossRevenuePayout': {
            'name': '总输赢',
            'total': grossRevenuePayout_total,
            'rabateCash': grossRevenuePayout_affi_1,
            'rabateCashPercent': grossRevenuePayout_affi_p
        },
        'stake': {
            'name': '总流水',
            'total': stake_total,
            'rabateCash': stake_affi_1,
            'rabateCashPercent': stake_affi_p
        },
        'activeMember': {
            'name': '活跃人数',
            'total': activeMember_total,
            'rabateCash': activeMember_affi_1,
            'rabateCashPercent': activeMember_affi_p
        },
        'houseEdge': {
            'name': '杀数',
            'total': houseEdge_total,
            'rabateCash': houseEdge_affi_1,
            'rabateCashPercent': houseEdge_affi_p
        }
    }

    df = pd.DataFrame(columns=['项目', '官网', '佣金代理', '代理占比'])

    for d in summary_for_affi.values():
        d_name = d['name']
        d_total = d['total']
        d_rabateCash = d['rabateCash']
        d_rabateCashPercent = d['rabateCashPercent']

        df = df.append({
            '官网': d_total,
            '佣金代理': d_rabateCash,
            '代理占比': d_rabateCashPercent,
            '项目': d_name
        }, ignore_index=True)
    return df

In [230]:
test_affiliate_mixreport('2019-09-08', '2019-09-14')

,项目,官网,佣金代理,代理占比
0,总存款,1.353620e+08,5.187146e+07,0.383205
1,总提款,1.226284e+08,4.604828e+07,0.375511
2,存提差,1.273363e+07,5.823177e+06,0.457307
3,注册人数,6.510000e+02,2.350000e+02,0.360983
4,注册后存款人数,2.980000e+02,1.040000e+02,0.348993
5,总输赢,2.582246e+07,8.627102e+06,0.334093
6,总流水,1.084819e+09,3.775637e+08,0.348043
7,活跃人数,7.268000e+03,2.637000e+03,0.362823
8,杀数,2.380348e-02,2.284939e-02,0.959918


In [3]:
# %%time
name = "Affiliate"
engine = utils.get_db("IDC", "LckAffiliatedb")
query = "SELECT * FROM [dbo].[{}] WITH (NOLOCK)".format(name)
affiliate = pd.read_sql(query, engine)
# engine.table_names()

In [4]:
query = "SELECT * FROM dbo.Member with (NOLOCK)"
member = utils.read_sql('IDC_ALL', 'LckBaseDB', query)

In [45]:
start = '2019-09-01'
end = '2019-09-24'

In [41]:
aff_money = affiliate.query('affitype == 1')
target_affid = aff_money.Id.unique()

In [39]:
member[member.Affiliate.isin(aff_money.Id)].query('RegDate >= @start & RegDate <= @end')

,MemberId,MemberName,Password,FirstName,LastName,Status,PaymentStatus,Affiliate,Email,IsEmail,MobilePhone,IsMobilePhone,RiskCategory,GroupId,Currency,OutstandingBalance,Balance,LastIp,LastVisit,LastDeposit,LastWithDrawal,LastTransaction,Token,AutoUpgrade,QuickTransfer,Verified,ReferrerId,RegIp,RegDate,PTSlotLastMsgTime,MSSlotLastMsgTime,PTSlotWallet,MSClubWallet,SeniorMember,IsOpenLargeCustomers,WithdrawalStatus,Source,IsDepositActive,IsWithdrawalActive,IsTransferActive,IsSmsNotifyActive,IsPushNotifyActive,VipLevel
484726,486576,358553741,11f98a73c4f8e89bcdd942afa1039cf4:9a,,,1,1,3395,,0,13601934368,1,0,1,RMB,0.0,0.00,180.174.216.32,2019-09-01 00:02:02.777,NaT,None,None,3102a5dc66f3ffa9c5518b67d097af15,1,1,0,0,180.174.216.32,2019-09-01 00:02:02.667,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484728,486578,qq157303,748f7b82c9f06f4185d324f19537e379:f9,,,1,1,1906,,0,18576673424,1,0,1,RMB,0.0,0.00,124.64.19.42,2019-09-01 01:36:32.937,NaT,None,None,a8689fbb39dff0952375eaaa11a211db,1,1,0,0,124.64.19.42,2019-09-01 01:36:32.860,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484731,486581,m023088,c1ae5b4d41c9588c38160e27f17d6dbd:55,,,1,1,2827,741473467@qq.com,0,15310789051,0,0,1,RMB,0.0,0.00,49.117.245.221,2019-09-01 02:22:11.803,NaT,None,None,0e7014b9869c54d0852c67accc461f20,1,1,0,0,49.117.245.221,2019-09-01 02:20:36.007,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484733,486583,tt16888,cd1e34d44b06f35db1dcefcb7f7e5895:78,,,1,1,1906,,0,18369651976,1,0,1,RMB,0.0,0.00,27.221.115.163,2019-09-01 03:02:53.903,NaT,None,None,210963335fb44c33afe5ad27caafb90e,1,1,0,0,27.221.115.163,2019-09-01 03:02:53.840,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484743,486593,hwz362411,511d9d1d14fcf9c99f2cf7874258dcd7:a8,黄文专,,1,1,3384,50338265@qq.com,0,13850701099,1,0,1,RMB,0.0,23.16,183.251.22.133,2019-09-10 21:23:15.587,NaT,None,None,b6c2858f964ad341fe7b995784a050eb,1,1,0,0,120.41.123.164,2019-09-01 10:53:38.597,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484745,486608,yao19857089117,ed07b846b62ec1ccabcaa221f938d634:5b,姚海燕,,1,1,1118,,0,19857089117,1,0,1,RMB,0.0,2.15,123.152.169.221,2019-09-23 14:58:24.893,NaT,None,None,391c2957c1a9ecc1c820a22e6c58a995,1,1,0,0,39.185.201.106,2019-09-01 14:30:51.930,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484750,486613,aaa4789,397de3c8493436c2f75ec6e7208bb78a:e3,,,1,1,2289,586787786@qq.com,0,13847767017,0,0,1,RMB,0.0,0.00,123.178.138.242,2019-09-01 15:00:32.173,NaT,None,None,66c0b3d635d5c1fe8dfb33736db39f9d,1,1,0,0,123.178.138.242,2019-09-01 15:00:35.153,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484751,486614,yuyiganlan,35731565287e1acdabf69e9fc3938d50:ec,梁嵩,,1,1,1471,2943939496@qq.com,0,13538032520,0,0,1,RMB,0.0,0.40,183.54.41.37,2019-09-22 22:11:58.487,NaT,None,None,a399b8cb2da09862628dae6f2c646b8c,1,1,0,0,113.107.67.46,2019-09-01 15:22:37.013,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484753,486616,maliang03,dd80f6254770878c0a6dd77546acd671:32,马亮,,1,1,3455,,0,15769685867,1,0,1,RMB,0.0,24.23,111.49.35.152,2019-09-18 22:46:45.423,NaT,None,None,738668f3cb7abc3ef4c568d6c5be9d62,1,1,0,0,223.104.29.192,2019-09-01 15:34:02.680,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0
484761,486624,lin1995,58a0caf9cec9e38036d2c684e03337eb:84,张琳,,1,1,1906,lin15546687131@sina.com,1,15546687131,1,0,1,RMB,0.0,0.05,113.5.5.27,2019-09-20 18:44:47.097,NaT,None,None,33e198e2aae30da3efebe6be02c439f7,1,1,0,0,113.5.5.36,2019-09-01 16:10:25.390,NaT,None,0,1,False,False,True,0,True,True,True,True,True,0


In [10]:
aff_money.head(1)

,Id,Currency,RateGroup,MemberId,MemberName,AffiName,LoginName,Password,WebSite,LastTime,LastIP,IsMoney,Status,AffiUrl,AddTime,AddBy,GrantLastTime,TranDate,TranIndex,WinLossDate,WinLossIndex,LastPay,Remark,affitype,Token,IsLockLogin,DefaultGrantRate,ParentAffId,PromoCode
11,1011,RMB,0,0,xxxx,第一足球,xxxx,,,2014-12-12,,0,1,,2014-12-12,,2014-12-12,2019-09-24,0,2019-09-24,0,2014-12-12,第一足球,1,None,False,NaN,None,BZAA


In [9]:
g_d_s_tmp = game_daily_summary.query('date >= @start & date < @end')

In [42]:
g_d_s_tmp.query('aff_id.isin(@target_affid)').groupby('product_type')['stake', 'gross_revenue_payout_time'].sum().reset_index()

,product_type,stake,gross_revenue_payout_time
0,AsiaGameAgin,5.343094e+08,7.887551e+06
1,AsiaGameHunter,2.827171e+06,2.015845e+05
2,AsiaGameXin,3.973777e+06,1.857496e+05
3,AsiaGameYoplay,3.876822e+07,-3.738156e+05
4,GoldDeluxCasino,1.702274e+07,2.783835e+05
5,IBOCasino,5.800000e+02,-2.220000e+02
6,IMCasino,4.560921e+06,-2.261790e+04
7,IMESport,3.571918e+06,1.678692e+05
8,IMKYCard,1.426465e+07,5.973912e+05
9,IMLYCard,2.272891e+07,6.126061e+05


In [6]:
game_daily_summary.query('date >= @start & date < @end & product_type == "NewSport188"')['stake', 'gross_revenue_payout_time'].sum().reset_index()

,member_name,date,product_type,stay_hours,gross_revenue_bet_time,gross_revenue_payout_time,stake,bet_counts,royalty,rebate,member_id,aff_id,gender,birthday,age,zodiac,client_type,is_deposited
0,00003598,2018-11-07,NewSport188,5.0,250.0,250.0,1100.0,11.0,32.5,4.4,453419,1990,0.0,1987-04-12,32.0,牡羊,3,True


In [43]:
member_profile.head(1)

,member_id,member_name,status,group_id,balance,reg_date,aff_id,is_aff,is_push_active,country,province,longitude,latitude,gender,birthday,age,zodiac,deposit_days,deposit_oldest_date,deposit_latest_date,deposit_counts,deposit_daily_max,deposit_daily_median,deposit_daily_min,deposit_amount,...,withdraw_amount,withdraw_hour_median,gross_revenue_amount,gross_revenue_max,gross_revenue_min,gross_revenue_std,gross_revenue_median,stake_amount,stake_max,stake_min,stake_std,stake_median,royalty,rebate,bet_counts,bet_oldest_date,bet_latest_date,bet_days,prefer_game,first_game,prefer_game_type,first_game_type,client_type,is_direct,cash_loss
0,1,demormb8888,2,1,0.0,2009-08-11,0,0,True,,,9999.0,9999.0,1.0,1990-02-11,29.0,水瓶,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,NaN,1.0,0.0


In [50]:
tt = member_summary.query('Date >= @start & Date < @end & DepositAmount > 0').MemberId.unique()

In [52]:
member_profile.query('reg_date >= @start & reg_date < @end & aff_id != 0 & status == 1 & member_id.isin(@tt) & aff_id.isin(@target_affid)').member_id.nunique()

464

In [19]:
member_profile[member_profile.aff_id.isin(aff_money.Id)].query('reg_date >= @start & reg_date <= @end').member_id.nunique()

963

In [54]:
member_profile[member_profile.aff_id.isin(aff_money.Id)].query('reg_date >= @start & reg_date <= @end & deposit_oldest_date <= @end & status == 1').member_id.nunique()

464

In [29]:
member_profile.tail()

,member_id,member_name,status,group_id,balance,reg_date,aff_id,is_aff,is_push_active,country,province,longitude,latitude,gender,birthday,age,zodiac,deposit_days,deposit_oldest_date,deposit_latest_date,deposit_counts,deposit_daily_max,deposit_daily_median,deposit_daily_min,deposit_amount,...,withdraw_amount,withdraw_hour_median,gross_revenue_amount,gross_revenue_max,gross_revenue_min,gross_revenue_std,gross_revenue_median,stake_amount,stake_max,stake_min,stake_std,stake_median,royalty,rebate,bet_counts,bet_oldest_date,bet_latest_date,bet_days,prefer_game,first_game,prefer_game_type,first_game_type,client_type,is_direct,cash_loss
486687,488528,mkxax99,1,1,0.0,2019-09-17,0,0,True,中国,安徽,117.718470,29.857230,0.0,1984-05-12,35.0,金牛,1.0,2019-09-17,2019-09-17,1.0,500.0,500.0,500.0,500.0,...,192.0,24.0,307.5,307.5,307.5,NaN,307.5,1092.5,1092.5,1092.5,NaN,1092.5,24.60,8.740,22.0,2019-09-17,2019-09-17,1.0,AsiaGameAgin,AsiaGameAgin,真人,真人,1.0,1.0,308.0
486688,488529,zbh5520,1,1,8.0,2019-09-17,0,0,True,中国,重庆,106.504962,29.533155,0.0,1993-07-13,26.0,巨蟹,1.0,2019-09-17,2019-09-17,1.0,808.0,808.0,808.0,808.0,...,0.0,NaN,-67.0,-67.0,-67.0,NaN,-67.0,127.0,127.0,127.0,NaN,127.0,-5.36,1.016,5.0,2019-09-17,2019-09-17,1.0,AsiaGameAgin,AsiaGameAgin,真人,真人,1.0,1.0,808.0
486689,488530,xl440q,1,1,0.0,2019-09-17,1573,0,True,中国,青海,101.778916,36.623178,0.0,1988-04-01,31.0,牡羊,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,1.0,0.0,0.0
486690,488531,q13931433359,1,1,0.0,2019-09-17,2911,0,True,中国,河北,114.884091,40.811901,0.0,1970-01-01,49.0,摩羯,1.0,2019-09-17,2019-09-17,1.0,100.0,100.0,100.0,100.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,2.0,0.0,100.0
486691,488532,kahar02,1,1,0.0,2019-09-17,2832,0,True,中国,新疆,87.617733,43.792818,0.0,1970-01-01,49.0,摩羯,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,None,None,None,None,2.0,0.0,0.0


#### 存提款比對 9/01~9/23(含)

In [108]:
start_date = '2019-09-01'
end_date = '2019-09-01'

In [109]:
affiliate['is_money'] = 1
obs_aff = affiliate.query('affitype == 1')[['Id', 'is_money']]

In [110]:
obs_m_s = member_summary.query(
    'Date >= "{0}" & Date <= "{1}"'.format(start_date, end_date))
obs_m_s_affi_1 = obs_m_s.merge(obs_aff.rename(
    columns={'Id': 'AffiliateId'}), on='AffiliateId', how='left').query('is_money == 1')
obs_g_d_s = game_daily_summary.query(
    'date >= "{0}" & date <= "{1}"'.format(start_date, end_date))
obs_g_d_s_affi_1 = obs_g_d_s.merge(obs_aff.rename(
    columns={'Id': 'aff_id'}), on='aff_id', how='left').query('is_money == 1')

In [112]:
depositAmount_total = obs_m_s.DepositAmount.sum()
depositAmount_affi_1 = obs_m_s_affi_1.DepositAmount.sum()
depositAmount_affi_p = depositAmount_affi_1/depositAmount_total

# 總提款
withdrawalAmount_total = obs_m_s.WithdrawalAmount.sum()
withdrawalAmount_affi_1 = obs_m_s_affi_1.WithdrawalAmount.sum()
withdrawalAmount_affi_p = withdrawalAmount_affi_1/withdrawalAmount_total

# 存提差(存款-提款)
deposit_withdrawal_diff_toal = depositAmount_total - withdrawalAmount_total
deposit_withdrawal_diff_affi_1 = depositAmount_affi_1 - withdrawalAmount_affi_1
deposit_withdrawal_diff_p = deposit_withdrawal_diff_affi_1/deposit_withdrawal_diff_toal


# 筆數呢
d_count = obs_m_s_affi_1.DepositCount.sum()
w_count = obs_m_s_affi_1.WithdrawalCount.sum()




In [113]:
w_count,d_count

(1207.0, 2632.0)

#### 存款比較資料差在哪

In [158]:
pd.options.display.max_rows = 200
tmp = obs_m_s_affi_1
tmp.AffiliateId = tmp.AffiliateId.astype(int)
tmp = tmp.query('DepositCount > 0').groupby('AffiliateId')['MemberId', 'DepositCount', 'DepositAmount'].agg({'DepositCount':'sum', 'MemberId':'nunique', 'DepositAmount':'sum'}).reset_index()

In [159]:
DB = pd.read_csv('Results1.csv')

In [160]:
tmp = tmp.rename(columns={'AffiliateId':'affid', 'DepositCount':'DepositCount_DS', 'MemberId':'DepositMember_DS', 'DepositAmount':'Deposit_DS'})

In [161]:
tmp = tmp.merge(DB, on = 'affid', how = 'outer')

In [141]:
# Deposit = ft.read_dataframe(path +'deposit.ft').reset_index(drop=True)

In [174]:
Deposit.query('TransferTime >= "2019-09-1" & TransferTime < "2019-09-02" & MemberId == 165147')

,DepositId,TransactionId,MemberId,MemberName,DepositAmount,BankNo,ClientType,AppType,TransferTime,UpdateTime
8531065,14947539,PDB190901060756ha6,165147,cj9418,10000.0,6217856000085337698,3,1.0,2019-09-01 06:07:36.100,2019-09-01 06:09:14.437
8535233,14952801,PDB1909012024ye12i,165147,cj9418,2000.0,6217002970110502628,3,1.0,2019-09-01 20:24:51.007,2019-09-01 20:26:40.113
8535277,14952857,PDB19090120306tzpf,165147,cj9418,4500.0,6217002970110502628,3,1.0,2019-09-01 20:30:57.307,2019-09-01 20:32:49.957
8535307,14952896,PDB1909012034r5j7y,165147,cj9418,15000.0,6217002970110502628,3,1.0,2019-09-01 20:34:41.703,2019-09-01 20:36:22.140
8535340,14952932,PDB1909012039mj0i0,165147,cj9418,30000.0,6217002970110502628,3,1.0,2019-09-01 20:39:30.667,2019-09-01 20:40:30.227
8535366,14952961,PDB1909012042r1hgh,165147,cj9418,50000.0,6228481379455844479,3,1.0,2019-09-01 20:42:36.070,2019-09-01 20:44:16.663
8535388,14952992,PDB19090120452c85o,165147,cj9418,30000.0,6228481379455844479,3,1.0,2019-09-01 20:45:51.100,2019-09-01 20:46:48.650
8535455,14953083,PDB1909012055mqxzl,165147,cj9418,50000.0,6228481379455844479,3,1.0,2019-09-01 20:55:32.573,2019-09-01 20:57:12.340
8535501,14953137,PDB19090121010lwnh,165147,cj9418,5000.0,6226201103648308,3,1.0,2019-09-01 21:01:43.803,2019-09-01 21:03:45.550


In [183]:
obs_m_s_affi_1.query('DepositCount > 0 & AffiliateId.isin(@qq)').MemberId.unique()

array([155803, 165147, 167397, 170117, 170621, 181034, 183413, 199055,
       227792, 228226, 240375, 244047, 245778])

In [182]:
qq = tmp.query('DepositMember.isnull()').affid

# 確定有哪些遊戲資料

In [ ]:
c_affiliate, c_member_profile, c_game_daily_summary, c_member_summary, start_date, end_date

In [ ]:
summary_for_affi = ''
obs_aff = c_affiliate.query('affitype == 1')[
    ['Id', 'is_money']]

# 資料篩選處理
obs_m_s = c_member_summary.query(
    'Date >= "{0}" & Date <= "{1}"'.format(start_date, end_date))
obs_m_s_affi_1 = obs_m_s.merge(obs_aff.rename(
    columns={'Id': 'AffiliateId'}), on='AffiliateId', how='left').query('is_money == 1')
obs_g_d_s = c_game_daily_summary.query(
    'date >= "{0}" & date <= "{1}"'.format(start_date, end_date))
obs_g_d_s_affi_1 = obs_g_d_s.merge(obs_aff.rename(
    columns={'Id': 'aff_id'}), on='aff_id', how='left').query('is_money == 1')

# 總存款
depositAmount_total = obs_m_s.DepositAmount.sum()
depositAmount_affi_1 = obs_m_s_affi_1.DepositAmount.sum()
depositAmount_affi_p = depositAmount_affi_1/depositAmount_total

# 總提款
withdrawalAmount_total = obs_m_s.WithdrawalAmount.sum()
withdrawalAmount_affi_1 = obs_m_s_affi_1.WithdrawalAmount.sum()
withdrawalAmount_affi_p = withdrawalAmount_affi_1/withdrawalAmount_total

# 存提差(存款-提款)
deposit_withdrawal_diff_toal = depositAmount_total - withdrawalAmount_total
deposit_withdrawal_diff_affi_1 = depositAmount_affi_1 - withdrawalAmount_affi_1
deposit_withdrawal_diff_p = deposit_withdrawal_diff_affi_1/deposit_withdrawal_diff_toal

# 註冊人數
registerMember_total = c_member_profile.query(
    'reg_date >= "{0}" & reg_date <= "{1}" & group_id != 15'.format(start_date, end_date)).shape[0]
registerMember_affi_1 = c_member_profile.query(
    'reg_date >= "{0}" & reg_date <= "{1}" & aff_id > 0 & group_id != 15'.format(start_date, end_date)).shape[0]
registerMember_affi_p = registerMember_affi_1/registerMember_total

# 註冊後存款人數
registerAndDepositMember_total = c_member_profile.query(
    'reg_date >= "{0}" & reg_date <= "{1}" & group_id != 15 & deposit_counts > 0'.format(start_date, end_date)).shape[0]
registerAndDepositMember_affi_1 = c_member_profile.query(
    'reg_date >= "{0}" & reg_date <= "{1}" & aff_id > 0 & group_id != 15 & deposit_counts > 0'.format(start_date, end_date)).shape[0]
registerAndDepositMember_affi_p = registerAndDepositMember_affi_1 / \
    registerAndDepositMember_total

# 總輸贏
grossRevenuePayout_total = obs_g_d_s.gross_revenue_payout_time.sum()
grossRevenuePayout_affi_1 = obs_g_d_s_affi_1.gross_revenue_payout_time.sum()
grossRevenuePayout_affi_p = grossRevenuePayout_affi_1/grossRevenuePayout_total

# 總流水
stake_total = obs_g_d_s.stake.sum()
stake_affi_1 = obs_g_d_s_affi_1.stake.sum()
stake_affi_p = stake_affi_1/stake_total

# 活躍人數
activeMember_total = obs_g_d_s.member_id.nunique()
activeMember_affi_1 = obs_g_d_s_affi_1.member_id.nunique()
activeMember_affi_p = activeMember_affi_1/activeMember_total

# 殺數
houseEdge_total = obs_g_d_s.gross_revenue_payout_time.sum()/obs_g_d_s.stake.sum()
houseEdge_affi_1 = obs_g_d_s_affi_1.gross_revenue_payout_time.sum() / \
    obs_g_d_s_affi_1.stake.sum()
houseEdge_affi_p = houseEdge_affi_1/houseEdge_total